In [1]:
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence



import sys
if ".." not in sys.path:
    sys.path.insert(0, "..")

from datasets import *

dataset = PlayByPlayDataset("../data/nfl-big-data-bowl-2024/play_by_play_val.pkl")


torch.__version__


'2.1.0'

In [2]:
dataset[0].keys()


dict_keys(['offense_geometric', 'defense_geometric', 'offense_raw', 'defense_raw', 'ball_carrier_raw', 'tacklers_raw', 'game_id', 'play_id', 'player_tracking', 'event_timeseries', 'players_on_the_field', 'play_features', 'play_features_encoded', 'tackle_successful', 'yards_after_contact'])

In [5]:

dataloader = DataLoader(dataset, batch_size=4, shuffle=False, collate_fn=collate_padded_play_data_with_carrier_tackler_info)
batch = next(iter(dataloader))


In [14]:
batch["time_series_features"][3].size()


torch.Size([4])

In [4]:
test_batch_size = 4
sample_batchdict = create_batchdict([dataset[i] for i in range(test_batch_size)])

[t.view(t.size(0), -1, 6).size() for t in sample_batchdict["tacklers_raw"]]


[torch.Size([43, 1, 6]),
 torch.Size([21, 2, 6]),
 torch.Size([16, 1, 6]),
 torch.Size([47, 1, 6])]

In [16]:
(batch['time_series_features'][0].size(),
batch['time_series_features'][1].size(),
batch['time_series_features'][2].size(),
batch['time_series_features'][3],
)


(torch.Size([4, 47, 105]),
 torch.Size([4, 47, 6]),
 torch.Size([4, 47, 2, 6]),
 tensor([1, 2, 1, 1]))

In [59]:
# 248 is the dimensionality of the time-series features

encoder_layer = nn.TransformerEncoderLayer(d_model=248, nhead=8, batch_first=True)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
X = batch["time_series_features"].float()
out = transformer_encoder(X, src_key_padding_mask=(X[..., 0] == PAD_VALUE))


In [60]:
out.size() # batch size, time, n_features
# in the future -- concat with batch_size, n_features' shaped things -- need to squish out the time dim


torch.Size([8, 39, 248])

In [61]:
39 * 248


9672